# Lodders Solar Composition

Here we show how to work with the `LoddersComposition` class, which initializes a composition from the present day solar abundances of [Lodders et al. 2020 & 2021](https://sites.wustl.edu/planetarychemistrylaboratory/data-tables/). This class gives the option to set a desired metallicity `z=0.0` and scales the Lodders abundances accordingly. If no metallicity is provided, it will be calculated from the Lodders data as `(1 - H - He)`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from ipywidgets import interact, FloatSlider

In [ ]:
from pynucastro import networks
from pynucastro.nucdata import Nucleus

Scaling of H & He for a given Z:

In [ ]:
def xyz_from_comp(comp):
    x = 0.0
    y = 0.0
    z = 0.0

    for nuc, X_i in comp.items():
        if nuc.Z == 1: # Total H
            x += X_i
        elif nuc.Z == 2: # Total He
            y += X_i
        elif nuc.Z >= 3: # Total metals
            z += X_i

    return x, y, z

solar = networks.LoddersComposition()
x_solar, y_solar, z_solar = xyz_from_comp(solar)

print("Default solar metalicity is Z = ", z_solar)

def plot_lodders(z):

    comp = networks.LoddersComposition(z=z)

    x, y , z = xyz_from_comp(comp)

    plt.figure(figsize=(4,4))

    labels = ["H", "He", "Metals"]
    values = [x,y,z]

    x = np.arange(len(labels))

    fig, ax = plt.subplots()

    ax.bar(x, values)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylabel("Mass Fractions")
    ax.set_ylim(0, 1.0)
    ax.set_title(f"Lodders Composition at Z = {z:.3f}")

    plt.show()

interact(
    plot_lodders,
    z=FloatSlider(
        value=z_solar,
        min=0.01,
        max=0.04,
        step=0.001,
        description="Z",
        readout_format=".3f"
    )
)

Finding the top 10 most abundant isotopes in the Lodders composition:

In [ ]:
def top_10_isotopes(comp, n=10):
    fracs = sorted(comp.items(), key=lambda frac: frac[1], reverse=True)
    return fracs[:n]

solar = networks.LoddersComposition()
top = top_10_isotopes(solar, n=10)

for nuc, X in top:
    print(f"{nuc.short_spec_name:>6} X= {X:.4f}")

Binning Lodders composition to a network that only contains the top 10 abundant isotopes using `bin_as()`:

In [ ]:
solar = networks.LoddersComposition()

new_comp = [Nucleus("p"),
           Nucleus("h4"),
           Nucleus("o16"),
           Nucleus("c12"),
           Nucleus("ne20"),
           Nucleus("fe56"),
           Nucleus("n14"),
           Nucleus("si28"),
           Nucleus("mg24"),
           Nucleus("s32")]

binned_comp = solar.bin_as(new_comp, verbose=True)

print("Sum of binned comp:", sum(binned_comp.values()))